In [35]:
import numpy as np

In [36]:
l = np.load('../data/processed/DictaSign/list_videos.npy')

In [37]:
l

array(['S2_T1_A11', 'S2_T1_B15', 'S2_T2_A11', 'S2_T2_B15', 'S2_T3_A11',
       'S2_T3_B15', 'S2_T4_A11', 'S2_T4_B15', 'S2_T5_A11', 'S2_T5_B15',
       'S2_T6_A11', 'S2_T6_B15', 'S2_T7_A11', 'S2_T7_B15', 'S2_T9_A11',
       'S2_T9_B15', 'S3_T1_A2', 'S3_T1_B0', 'S3_T3_A2', 'S3_T3_B0',
       'S3_T4_A2', 'S3_T4_B0', 'S3_T5_A2', 'S3_T5_B0', 'S3_T6_A2',
       'S3_T6_B0', 'S3_T7_A2', 'S3_T7_B0', 'S3_T8_A2', 'S3_T8_B0',
       'S3_T9_A2', 'S3_T9_B0', 'S4_T1_A1', 'S4_T1_B14', 'S4_T2_A1',
       'S4_T2_B14', 'S4_T3_A1', 'S4_T3_B14', 'S4_T4_A1', 'S4_T4_B14',
       'S4_T5_A1', 'S4_T5_B14', 'S4_T6_A1', 'S4_T6_B14', 'S4_T7_A1',
       'S4_T7_B14', 'S4_T8_A1', 'S4_T8_B14', 'S4_T9_A1', 'S4_T9_B14',
       'S5_T1_A9', 'S5_T1_B17', 'S5_T3_A9', 'S5_T3_B17', 'S5_T4_A9',
       'S5_T4_B17', 'S5_T9_A9', 'S5_T9_B17', 'S6_T5_A6', 'S6_T5_B13',
       'S6_T6_A6', 'S6_T6_B13', 'S6_T8_A6', 'S6_T8_B13', 'S6_T9_A6',
       'S6_T9_B13', 'S7_T2_A10', 'S7_T2_B16', 'S7_T7_A10', 'S7_T7_B16',
       'S7_T9_A10', 'S7_T

In [112]:
l[67]

'S7_T2_B16'

In [38]:
signerRefsDictaSign = np.array(['A11','B15','A2','B0','A1','B14','A9','B17','A6','B13','A10','B16','A7','B4','A3','B5'])

In [103]:
def getVideoIndicesSplitDictaSign(sessionsSplit,tasksSplit,signersSplit,from_notebook=False):
    """
        Train/valid/test split for DictaSign
        Returns intersection of sessions, tasks and signers

        Inputs:
            sessionsSplit: list of 3 lists of session indices (2 to 9) for train, valid, test
            tasksSplit: list of 3 lists of task indices (1 to 9) for train, valid, test
            signersSplit: list of 3 lists of signer indices (0 to 15) for train, valid, test
            from_notebook: if notebook script, data is in parent folder

        Outputs:
            idxTrain, idxValid, idxTest: numpy arrays
    """
    if from_notebook:
        parent = '../'
    else:
        parent = ''

    l = np.load(parent+'data/processed/DictaSign/list_videos.npy')
    nVideos = len(l)
    sess = []
    task = []
    signer = []
    for iV in range(nVideos):
        tmp = l[iV].replace('S','').replace('T','').split('_')
        sess.append(int(tmp[0]))
        task.append(int(tmp[1]))
        signer.append(signerRefToSignerIdxDictaSign(tmp[2]))

    sess = np.array(sess)
    task = np.array(task)
    signer = np.array(signer)

    idxTrainSession = np.ones(nVideos)
    idxTrainTask = np.ones(nVideos)
    idxTrainSigner = np.ones(nVideos)
    idxValidSession = np.ones(nVideos)
    idxValidTask = np.ones(nVideos)
    idxValidSigner = np.ones(nVideos)
    idxTestSession = np.ones(nVideos)
    idxTestTask = np.ones(nVideos)
    idxTestSigner = np.ones(nVideos)

    for i in range(3):
        if len(sessionsSplit[i])==0:
            sessionsSplit[i] = [j for j in range(2,10)]
        if len(tasksSplit[i])==0:
            tasksSplit[i] = [j for j in range(1,10)]
        if len(signersSplit[i])==0:
            signersSplit[i] = [j for j in range(0,16)]

    for sessionTrain in sessionsSplit[0]:
        idxTrainSession[sess == sessionTrain] = 0
    for taskTrain in tasksSplit[0]:
        idxTrainTask[task == taskTrain] = 0
    for signerTrain in signersSplit[0]:
        idxTrainSigner[signer == signerTrain] = 0
    idxTrainSession = 1-idxTrainSession
    idxTrainTask = 1-idxTrainTask
    idxTrainSigner = 1-idxTrainSigner
    idxTrain = idxTrainSession*idxTrainTask*idxTrainSigner

    for sessionValid in sessionsSplit[1]:
        idxValidSession[sess == sessionValid] = 0
    for taskValid in tasksSplit[1]:
        idxValidTask[task == taskValid] = 0
    for signerValid in signersSplit[1]:
        idxValidSigner[signer == signerValid] = 0
    idxValidSession = 1-idxValidSession
    idxValidTask = 1-idxValidTask
    idxValidSigner = 1-idxValidSigner
    idxValid = idxValidSession*idxValidTask*idxValidSigner

    for sessionTest in sessionsSplit[2]:
        idxTestSession[sess == sessionTest] = 0
    for taskTest in tasksSplit[2]:
        idxTestTask[task == taskTest] = 0
    for signerTest in signersSplit[2]:
        idxTestSigner[signer == signerTest] = 0
    idxTestSession = 1-idxTestSession
    idxTestTask = 1-idxTestTask
    idxTestSigner = 1-idxTestSigner
    idxTest = idxTestSession*idxTestTask*idxTestSigner

    return idxTrain, idxValid, idxTest

In [104]:
getVideoIndicesSplitDictaSign([[2,3],[4,5],[6,7]],[[1,2,4,5],[6],[7]],[[0],[1],[10]],from_notebook=True)

(array([1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [100]:
sessionsSplit=[[2,3],[4,5],[6,7]]
tasksSplit=[[1,2,4,5],[6],[7]]
signersSplit=[[0],[1],[2]]

In [56]:
if True:
    parent = '../'
else:
    parent = ''

l = np.load(parent+'data/processed/DictaSign/list_videos.npy')
nVideos = len(l)
sess = []
task = []
signer = []
for iV in range(nVideos):
    tmp = l[iV].replace('S','').replace('T','').split('_')
    sess.append(int(tmp[0]))
    task.append(int(tmp[1]))
    signer.append(signerRefToSignerIdxDictaSign(tmp[2]))

sess = np.array(sess)
task = np.array(task)
signer = np.array(signer)
print(sess)
print(task)
print(signer)

idxTrain = np.ones(nVideos)
idxValid = np.ones(nVideos)
idxTest = np.ones(nVideos)

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 7 7 7 7 7 7 8 8
 8 8 8 8 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9]
[1 1 2 2 3 3 4 4 5 5 6 6 7 7 9 9 1 1 3 3 4 4 5 5 6 6 7 7 8 8 9 9 1 1 2 2 3
 3 4 4 5 5 6 6 7 7 8 8 9 9 1 1 3 3 4 4 9 9 5 5 6 6 8 8 9 9 2 2 7 7 9 9 2 2
 7 7 8 9 1 1 2 2 3 3 4 4 5 5 6 6 8 8 9 9]
[ 0  1  0  1  0  1  0  1  0  1  0  1  0  1  0  1  2  3  2  3  2  3  2  3
  2  3  2  3  2  3  2  3  4  5  4  5  4  5  4  5  4  5  4  5  4  5  4  5
  4  5  6  7  6  7  6  7  6  7  8  9  8  9  8  9  8  9 10 11 10 11 10 11
 12 13 12 13 12 12 14 15 14 15 14 15 14 15 14 15 14 15 14 15 14 15]


In [101]:
sessionsTrain=[2,3,4,5,6,9]
sessionsValid=[8]
sessionsTest=[7] # session 7
tasksTrain=[]
tasksValid=[]
tasksTest=[]
signersTrain=[]
signersValid=[]
signersTest=[]

In [106]:
idxTrain, idxValid, idxTest = getVideoIndicesSplitDictaSign([sessionsTrain,sessionsValid,sessionsTest],
                                                            [tasksTrain,tasksValid,tasksTest],
                                                            [signersTrain,signersValid,signersTest],True)

In [107]:
idxTrain

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [108]:
idxValid

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [109]:
idxTest

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
       1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [111]:
np.where(idxTest)[0]

array([66, 67, 68, 69, 70, 71])